<a href="https://colab.research.google.com/github/software-artisan/tsai_end3/blob/main/session_2_5_pytorch_101/session_3_assignment_digit_recognition_addition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Handwritten digit Recognition and adder network



In [37]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

torch.__version__
torch.cuda.get_device_name()
torch.cuda.get_device_properties('cuda:0')

_CudaDeviceProperties(name='Tesla P100-PCIE-16GB', major=6, minor=0, total_memory=16280MB, multi_processor_count=56)

In [26]:
once = True

# this class defines the CNN or convulational neural network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # shown below are the definitions of the layers of the network

        # input = 28x28x1 | kernels = (3x3x1)x8 | output = 28x28x8  | RF = 3x3
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3, padding=1) #input -? OUtput? RF
        
        # input = 28x28x8 (padding=1) | kernels = (3x3x32)x16 | output = 28x28x16 | RF = 5x5
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, padding=1)
        
        # input = 28x28x16 | maxpool = 2x2 | output = 14x14x16 | RF = 10x10
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # input = 14x14x16 (padding=1) | kernels = (3x3x64)16 | output = 14x14x16 | RF = 12x12
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3, padding=1)
        
        # input = 14x14x16 (padding=1) | kernels = (3x3x128)x16 | output = 14x14x16 | RF = 14x14
        self.conv4 = nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3, padding=1)
        
        # input = 14x14x16 | maxpool = 2x2 | output = 7x7x16 | RF = 28x28
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # # input = 7x7x64 | kernel = (3x3x256)x128 | output = 5x5x128 | RF=30x30
        # self.conv5 = nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3)
        
        # # input = 5x5x128 | kernel = (3x3x512)x256 | output = 3x3x256 | RF=32x32
        # self.conv6 = nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3)
        
        # # input = 3x3x256 | kernel = (3x3x1024)x10 | output = 1x1x10 | RF=34x34
        # self.conv7 = nn.Conv2d(in_channels=16, out_channels=10, kernel_size=3)

        # input = 794 | output = 120 
        self.fc1 = nn.Linear(in_features=7*7*16 + 10, out_features=120)
        # input = 120 | output = 60
        self.fc2 = nn.Linear(in_features=120, out_features=60)

        # this is the output layer for mnist
        # input = 60 | output = 10
        self.out_mnist = nn.Linear(in_features=60, out_features=10)

        # this is the output layer for the adder
        # input=60 | output=19 (max sum of two digits=18)
        self.out_adder = nn.Linear(in_features=60, out_features=19)

    def forward(self, mnist, num):
        global once

        # seems to define the forward propogation of the neural network
        mnist = self.pool1(F.relu(self.conv2(F.relu(self.conv1(mnist)))))
        mnist = self.pool2(F.relu(self.conv4(F.relu(self.conv3(mnist)))))

        # x = F.relu(self.conv6(F.relu(self.conv5(x))))
        # #x = F.relu(self.conv7(x))
        # x = self.conv7(x)
        # x = x.view(-1, 10)
        # return F.log_softmax(x)

        #if once: print(f"mnist.shape after maxpool2={mnist.shape}")
        mnist = mnist.reshape(-1, 7*7*16)
        # Concatenating one hot encoding of input2 digit to FC layer
        mnist_and_num = torch.cat((mnist, num), dim=1) 
        #if once: print(f"mnist_and_num.shape after torch.cat()={mnist_and_num.shape}")
        # hidden fc1 linear layer
        mnist_and_num = F.relu(self.fc1(mnist_and_num))

        # hidden fc2 linear layer
        mnist_and_num = F.relu(self.fc2(mnist_and_num))

        # output linear layer
        mnist = self.out_mnist(mnist_and_num)
        mnist_num_sum = self.out_adder(mnist_and_num)

        return F.log_softmax(mnist), F.log_softmax(mnist_num_sum)
        #return mnist, mnist_num_sum

In [34]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
# print a summary of the model for an input of size 28x28x1 (l x w xchannels)
#summary(model, input_size=[(1, 28, 28),(1,10)])
print(model)

Net(
  (conv1): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=794, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=60, bias=True)
  (out_mnist): Linear(in_features=60, out_features=10, bias=True)
  (out_adder): Linear(in_features=60, out_features=19, bias=True)
)


In [18]:

torch.manual_seed(1)
batch_size = 1000

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
# data loader for training ???.  download=True means download the dataset (training images and labels for images) if the dataset wasn't downloaded already
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))  # need to understand why we normalize?  even without normalization, accuracy was still 97%
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
# data loader for testing???  download=True means download the dataset (training images and labels for images) if the dataset wasn't downloaded already
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [27]:
# Decorate an iterable object, returning an iterator which acts exactly like the original iterable, but prints a dynamically updating progressbar every time a value is requested.
from tqdm import tqdm
import IPython.core.debugger

# this is the function that trains the specified 'model' using the 'train_loader', optimizer and others
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    epoch_loss = 0; epoch_correct = 0; epoch_adder_loss=0; epoch_sum_correct = 0; epoch_mnist_loss=0; epoch_mnist_correct=0
    for batch_idx, (data, target) in enumerate(pbar):  # target is the 'label' for this image..
        data, target = data.to(device), target.to(device)

        # generate random ints
        # Returns a tensor filled with random integers generated uniformly between low (inclusive) and high (exclusive).
        rand_digits = torch.randint(low=0, high=10, size=(batch_size,)) # batch_size rand ints
        # Takes LongTensor with index values of shape (*) and returns a tensor of shape (*, num_classes) that have zeros everywhere except where the index of last dimension matches the corresponding value of the input tensor, in which case it will be 1.
        rand_digits_one = F.one_hot(rand_digits, num_classes=10) # one hot encoding
        rand_digits_one = rand_digits_one.to(device)
        rand_digits = rand_digits.to(device)        

        optimizer.zero_grad()  # for each batch, zero the gradient (from processing of prior batch)
        
        output, summation = model(data, rand_digits_one)   # predict using the model for the given data (batch); for each batch, output.shape=torch.Size([128, 10]); for last batch it is output.shape=torch.Size([96, 10])
        
        #if (batch_idx in (0, 468)): 
        #  print(f"\noutput.shape={output.shape}; prediction for first image in batch: output[0]={output[0]}")  #print 'output' from 1st and last
        
        # The negative log likelihood loss.  The negative log likelihood loss. It is useful to train a classification problem with C classes. 
        # https://pytorch.org/docs/master/generated/torch.nn.functional.nll_loss.html
        mnist_loss = F.nll_loss(output, target)  # compute the loss
        adder_loss = F.nll_loss(summation, rand_digits + target) # Calculate Loss
        #mnist_loss = F.cross_entropy(output, target)  # compute the loss
        #adder_loss = F.cross_entropy(summation, rand_digits + target) # Calculate Loss

        loss = mnist_loss + adder_loss

        loss.backward()   # do backward propogation; update gradients of tensors in dynamic computation graph, starting from 'loss' node in the graph..
        
        optimizer.step()  # use the updated gradients to adjust the weight of each kernel
        
        pred_mnist = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability using argmax(); this index represents the predicted digit
        #  get the correct number of predictions for this batch
        mnist_correct = pred_mnist.eq(target.view_as(pred_mnist)).sum().item()   # tensor.eq() method: computes if one element of a tensor is equal to the corresponding element of another tensor;

        pred_sum = summation.argmax(dim=1, keepdim=True) # get the index of the max log-probability using argmax(); this index represents the sum of the two numbers
        sum_correct = pred_sum.eq((rand_digits + target).view_as(pred_sum)).sum().item()   # tensor.eq() method: computes if one element of a tensor is equal to the corresponding element of another tensor;  
        #IPython.core.debugger.set_trace()

        epoch_mnist_loss += mnist_loss
        epoch_mnist_correct += mnist_correct
        epoch_adder_loss += adder_loss
        epoch_sum_correct += sum_correct

        pbar.set_description(desc= f'Training phase: batch_id={batch_idx}; total loss={loss.item()}; mnist loss={mnist_loss.item()}; adder loss={adder_loss.item()}; epoch_mnist_correct={epoch_mnist_correct}/{len(train_loader.dataset)}; epoch_sum_correct={epoch_sum_correct}/{len(train_loader.dataset)}')

    print(f'Training set: Average mnist loss: {epoch_mnist_loss/len(train_loader.dataset):.4f}; mnist Accuracy: {epoch_mnist_correct}/{len(train_loader.dataset)}   {100. * epoch_mnist_correct / len(train_loader.dataset):.4f}%; adder Accuracy: {epoch_sum_correct}/{len(train_loader.dataset)}   {100. * epoch_sum_correct / len(train_loader.dataset):.4f}%')

# this functions tests the trained model 'model' by comparing the predictions done by the trained model with the label for the image in the test data
def test(model, device, test_loader):
    model.eval()
    test_loss = 0; test_sum_loss = 0
    correct = 0; sum_correct = 0
    pbar = tqdm(test_loader)
    with torch.no_grad():   # during testing/inferencing, backprop is not needed; so disable autograd.
        for data, target in pbar:   # target is the 'label' for this image..
            data, target = data.to(device), target.to(device)

            # generate random ints
            # Returns a tensor filled with random integers generated uniformly between low (inclusive) and high (exclusive).
            rand_digits = torch.randint(low=0, high=10, size=(batch_size,)) # batch_size rand ints
            # Takes LongTensor with index values of shape (*) and returns a tensor of shape (*, num_classes) that have zeros everywhere except where the index of last dimension matches the corresponding value of the input tensor, in which case it will be 1.
            rand_digits_one = F.one_hot(rand_digits, num_classes=10) # one hot encoding
            rand_digits_one = rand_digits_one.to(device)
            rand_digits = rand_digits.to(device)        

            output, summation = model(data, rand_digits_one)   # make the prediction for the given test data
            
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            test_sum_loss += F.nll_loss(summation, rand_digits + target, reduction='sum').item()

            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability using argmax(); this index represents the predicted digit
            pred_sum = summation.argmax(dim=1, keepdim=True)  # get the index of the max log-probability using argmax(); this index represents the predicted digit

            #  get the correct number of predictions for this batch
            correct += pred.eq(target.view_as(pred)).sum().item()   # tensor.eq() method: computes if one element of a tensor is equal to the corresponding element of another tensor;
            sum_correct += pred_sum.eq((rand_digits + target).view_as(pred_sum)).sum().item()   # tensor.eq() method: computes if one element of a tensor is equal to the corresponding element of another tensor;  

            pbar.set_description(desc=f'Testing phase: total_test_loss={test_loss + test_sum_loss}; total_correct={correct}/{len(test_loader.dataset)}; total_sum_correct={sum_correct}/{len(test_loader.dataset)}')

    test_loss /= len(test_loader.dataset)

    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [28]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
#optimizer = optim.Adam(model.parameters(), lr=0.005)


once=True
for epoch in range(50):
    print(f"Epoch: {epoch+1}" )
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    once=False


Epoch: 1


  0%|          | 0/60 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:69: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
Training phase: batch_id=59; total loss=5.173794746398926; mnist loss=2.3016982078552246; adder loss=2.872096538543701; epoch_mnist_correct=6742/60000; epoch_sum_correct=5561/60000: 100%|██████████| 60/60 [00:08<00:00,  6.90it/s]


Training set: Average mnist loss: 0.0023; mnist Accuracy: 6742/60000   11.2367%; adder Accuracy: 5561/60000   9.2683%


Testing phase: total_test_loss=51676.59521484375; total_correct=1135/10000; total_sum_correct=1002/10000: 100%|██████████| 10/10 [00:01<00:00,  6.51it/s]


Test set: Average loss: 2.3011, Accuracy: 1135/10000 (11%)

Epoch: 2


...

Test set: Average loss: 0.0323, Accuracy: 9903/10000 (99%)

Epoch: 50


Training phase: batch_id=59; total loss=0.8966447114944458; mnist loss=0.02803477644920349; adder loss=0.8686099052429199; epoch_mnist_correct=59511/60000; epoch_sum_correct=44550/60000: 100%|██████████| 60/60 [00:09<00:00,  6.63it/s]


Training set: Average mnist loss: 0.0000; mnist Accuracy: 59511/60000   99.1850%; adder Accuracy: 44550/60000   74.2500%


Testing phase: total_test_loss=8044.346004486084; total_correct=9899/10000; total_sum_correct=7603/10000: 100%|██████████| 10/10 [00:01<00:00,  6.68it/s]

Test set: Average loss: 0.0338, Accuracy: 9899/10000 (99%)

